In [113]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MySimpleLinearRegressionClassifier,\
    MyKNeighborsClassifier,\
    MyDummyClassifier

import numpy as np

# PA4 Auto Classification
### Sam Berkson


### Data Preparation

To begin, we will seed our random generator to ensure that our results can be replecated.  We then load in our data from a .txt file to a MyPyTable() object.  We then discretize our MPG values on a 1-10 scale.  The last step is to select our common training and testing sets.

In [114]:
np.random.seed(0)

table = MyPyTable()
table.load_from_file("auto-data-removed-NA.txt")

counter = 0
for row in table.data:
    if row[0] <= 13:
        table.data[counter][0] = 1
    elif row[0] == 14:
        table.data[counter][0] = 2
    elif (row[0] == 15 or row[0] == 16):
        table.data[counter][0] = 3
    elif (row[0] <= 19 and row[0] >= 17):
        table.data[counter][0] = 4
    elif (row[0] <= 23 and row[0] >= 20):
        table.data[counter][0] = 5
    elif (row[0] <= 26 and row[0] >= 24):
        table.data[counter][0] = 6
    elif (row[0] <= 30 and row[0] >= 27):
        table.data[counter][0] = 7
    elif (row[0] <= 36 and row[0] >= 31):
        table.data[counter][0] = 8
    elif (row[0] <= 44 and row[0] >= 37):
        table.data[counter][0] = 9
    else:
        table.data[counter][0] = 10
    counter += 1

headers = table.get_column_names()
randomIndexes = []
rows = []

i = 0
while(i < 5):
    randomIndexes.append(np.random.randint(0, len(table.data)))
    i += 1

for index in randomIndexes:
    rows.append(table.data[index])

X_test = []
for row in rows:
    X_test.append(row[4])

X_train =[]
y_train = []

index = 0
for row in table.data:
    if table.data[index] not in randomIndexes:
        X_train.append(row[4])
        y_train.append(row[0])
    index += 1

print("Testing set and indexes")
labels_actual = []

index = 0
for row in table.data:
    if index in randomIndexes:
        print("Index", index, ":", index, ", Weight: ", row[4], ", Label: ", row[0])
        labels_actual.append(row[0])
    index += 1


Testing set and indexes
Index 47 : 47 , Weight:  3139.0 , Label:  4
Index 117 : 117 , Weight:  2158.0 , Label:  6
Index 172 : 172 , Weight:  3211.0 , Label:  4
Index 192 : 192 , Weight:  2164.0 , Label:  6
Index 251 : 251 , Weight:  2720.0 , Label:  6


### Linear Regression MPG Classifier

Our next step is to fit a linear regression line between the vehicle weight and the discretized MPG label.  We pass in all instances (excluding our 5 testing instances), and fit a least squares linear regression to the data.  We then predict the MPG label of each instance in our testing set.

In [115]:
regressor = MySimpleLinearRegressionClassifier(None)
regressor.fit(X_train, y_train)
MPG_linear_labels = regressor.predict(X_test)

print("Linear Regression Predicted MPG Label: ", MPG_linear_labels)
print("Actual MPG Label: ", labels_actual)

Linear Regression Predicted MPG Label:  [4, 5, 6, 6, 5]
Actual MPG Label:  [4, 6, 4, 6, 6]


As we can see, our linear regression didnt totally miss the mark.  We had an accuracy of 40%, correctly predicting the 1st and 4th instances.  

### K Nearest Neighbors MPG Classifier

Our next step is to use our k nearest neighbors algorithm to predict MPG given the number of cylinders, the car's weight, and the car's acceleration.  We pass in all of our instances (excluding our 5 testing instances), and then predict the MPG label of each instance in our testing set

In [116]:
kNN_X_train = []

index = 0
for row in table.data:
    if index in randomIndexes:
        instance = []
        instance.append(row[1])
        instance.append(row[4])
        instance.append(row[5])
        kNN_X_train.append(instance)
    index += 1

kNN = MyKNeighborsClassifier(n_neighbors = 5)
kNN.fit(kNN_X_train, y_train)
distances, indices = kNN.kneighbors(X_test)
kNN_predicted_labels = kNN.predict(X_test)

print("kNN Predicted MPG Label: ", kNN_predicted_labels)
print("Actual MPG Label: ", labels_actual)

kNN Predicted MPG Label:  [4, 4, 4, 4, 4]
Actual MPG Label:  [4, 6, 4, 6, 6]


Our accuracy for k nearest neighbors is the same as our linear regression, clocking in at 40%.  This time, we correctly predicted the 1st and 3rd instances' labels.

### Dummy MPG Classifier

Our next step is to use our dummy classifier to predict MPG labels.  Our X_train is irrelevant here, since we are only concerned with the most common MPG label.  Our test set remains the same.

In [117]:
dummy = MyDummyClassifier()
dummy.fit(kNN_X_train, y_train)
dummy_predicted_labels = dummy.predict(X_test)

print("Dummy Predicted MPG Label: ", kNN_predicted_labels)
print("Actual MPG Label: ", labels_actual)

Dummy Predicted MPG Label:  [4, 4, 4, 4, 4]
Actual MPG Label:  [4, 6, 4, 6, 6]


As it turns out, 4 is our most common label. This means, you guessed it, our accuracy again lands at 40%.  We correctly predicted the same instances as our kNN (1st and 3rd instances).

### Classifier Comparison